# 📦 LLM Response Explainer
This notebook walks through how to **read, interpret, and programmatically use the output** returned by large language models (LLMs) using popular libraries like Hugging Face Transformers.

## 🧾 Structure of LLM Responses
LLMs typically return a list of one or more dictionaries. Each dictionary contains the model’s prediction and relevant metadata.

The **keys** inside these dictionaries tell you:
- What the model *thinks* the label/class is (e.g., `label` or `answer`)
- How *confident* it is (`score`)
- Optionally, where the answer is located (e.g., `start`, `end`)

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")
response = classifier("This is fantastic!")
print(response)

### 🧾 Sample Output Format:
```python
[{'label': 'POSITIVE', 'score': 0.9987}]
```

📌 This output is wrapped in a list, so you must access the `[0]` element before using dictionary keys like `'label'` or `'score'`.

## 🔍 Step 1: Access Specific Fields
Use Python’s dictionary syntax to access values inside the response dictionary. This is important when you want to extract results for tables, JSON, CSV, or downstream workflows.

In [ ]:
label = response[0]['label']
confidence = round(response[0]['score'], 3)
print(f"Label: {label}, Confidence: {confidence}")

### ✅ Why This Matters
If you don’t extract values properly, you’ll either get an error or use incorrect data. For example, calling `response["label"]` without the `[0]` will throw a `TypeError`, since `response` is a list.

## 🔁 Step 2: Working with Batches (Multiple Inputs)
You can send a list of text inputs to the model and receive a list of outputs. The order of results matches the order of inputs — this is guaranteed.

In [ ]:
texts = ["I love this", "This is bad", "Maybe"]
responses = classifier(texts)
for text, resp in zip(texts, responses):
    print(f"{text} → {resp['label']} ({resp['score']:.2f})")

### 📘 Tips:
- Always use `zip()` to pair input texts with responses
- Loop over batches for scalable logging, export, or analytics
- Use `round(resp['score'], 2)` to simplify outputs for display

## 🧱 Step 3: Wrap the Logic into a Reusable Function
Functions let you reuse logic and clean your notebook. You’ll need this for:
- Logging model predictions
- Saving results to a file
- Combining with data pipelines

In [ ]:
def explain_response(text):
    result = classifier(text)[0]
    return {
        "text": text,
        "label": result['label'],
        "confidence": round(result['score'], 2)
    }

In [ ]:
explain_response("Great work!")

### 📌 How This Helps
- You can later wrap `explain_response()` inside a `.apply()` on a DataFrame
- You can extend it to save into a database or visualize in a UI

## 🧬 Step 4: Special Output Formats – QA Example
Some models return additional fields — for example, **question answering** includes the start and end position of the answer inside the context.

In [ ]:
qa = pipeline("question-answering")
qa_output = qa(
    question="Who teaches this course?",
    context="Maria teaches this ACSPRI course on LLMs."
)
print(qa_output)

### Sample Output:
```python
{'score': 0.98, 'start': 0, 'end': 5, 'answer': 'Maria'}
```
This means:
- The model found the answer `'Maria'`
- It’s 98% confident
- The answer starts at character 0 and ends at character 5 in the context

In [ ]:
# You can extract individual fields like this:
qa_output['answer'], round(qa_output['score'], 2)

📌 Start/end indices are especially useful if you're building UI tools that highlight the answer inside a paragraph.

## 🧠 Summary: How to Use LLM Output Properly
| Task                | Output Format                  | What You Extract         |
|---------------------|-------------------------------|---------------------------|
| Sentiment Analysis  | List of dicts                 | `label`, `score`          |
| Question Answering  | Single dict                   | `answer`, `score`, `span` |
| Translation         | List of dicts with `translation_text` |                       |
| Named Entity Recognition | List of entities        | `entity`, `word`, `score` |

✅ Always inspect structure with `print()` or `type()` before using the result.

From here, you’ll start combining outputs into classification logic or decision rules.